In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import ipywidgets as widgets
df = pd.read_excel(r'data_center\US_Superstore_data.xls')
pd.set_option('display.max_columns', None)
#print(df.info())
#print(df.describe())
df.head()

In [ ]:
list_product = df['Category'].unique()
print(list_product)
list_product_category = {'Furniture': 0, 'Office Supplies': 1, 'Technology': 2}
list_state = df['State'].unique()
print(list_state)
list_sub_category = df['Sub-Category'].unique()
print(list_sub_category)
#df['Category'] = df['Category'].map(list_product_category)

In [ ]:
print(len(df))
year_data = df.drop_duplicates()
print(len(year_data))

# Pas de duplicats
#Verification des données si null
missing_data = year_data.isnull()
print(missing_data.sum())

What is the time frame of this dataset?
Date in nanoseconde precision

Le traitement des données de type objet pourrait compromettre les analyses, on doit les convertir en données exploitable

In [ ]:
# Ajout de Profit Margin au dataFrame avec verification de Sales != de 0
df['Profit Margin'] = np.where(df['Sales'] != 0, (df['Profit'] / df['Sales']) * 100, 0)
df['Order Year'] = df['Order Date'].dt.year
#En int pour etre exploitable
df['Order Month'] = df['Order Date'].dt.month

#print(df.head())

df_profil_year_month = pd.DataFrame({'Profit Margin': np.where(df['Sales'] != 0, (df['Profit'] / df['Sales']) * 100, 0), 
                                     'Order Year' : df['Order Date'].dt.year,
                                     'Order Month': df['Order Date'].dt.month,
                                     'Sales': df['Sales'],
                                     'Category': df['Category'],
                                     'State': df['State']
                                     })

print(df_profil_year_month.head())

In [ ]:
monthly_sales_year = df_profil_year_month.groupby(['Order Year', 'Order Month'])['Sales'].sum().unstack(level=0)

plt.figure(figsize=(12, 7))
for year in monthly_sales_year.columns:
    plt.plot(monthly_sales_year.index, monthly_sales_year[year], marker='o', label=str(year))

plt.title('Total Sales by Month and Year')
plt.xlabel('Month')
plt.ylabel('Total Sales $')
plt.xticks(range(1, 13),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Year')
plt.grid(True)
plt.show()

In [ ]:
from IPython.display import display

# Liste des catégories uniques
categories = df_profil_year_month['Category'].unique().tolist()

# Dropdown pour sélectionner une catégorie
dropdown = widgets.Dropdown(
    options=['All'] + categories,
    value='All',
    description='Category:',
)

def plot_sales_by_category(selected_category):
    plt.figure(figsize=(10,6))
    
    # Filtrer selon la catégorie sélectionnée
    if selected_category == 'All':
        data_filtered = df_profil_year_month
    else:
        data_filtered = df_profil_year_month[df_profil_year_month['Category'] == selected_category]
    
    # Agréger les ventes par mois
    monthly_sales_year = data_filtered.groupby(['Order Year','Order Month'])['Sales'].sum().unstack(level=0)
    
    for year in monthly_sales_year.columns:
        plt.plot(monthly_sales_year.index, monthly_sales_year[year], marker='o', label=str(year))

    plt.title('Total Sales by Month and Year')
    plt.xlabel('Month')
    plt.ylabel('Total Sales $')
    plt.xticks(range(1, 13),
           ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    plt.legend(title='Year')
    plt.grid(True)
    plt.show()

widgets.interact(plot_sales_by_category, selected_category=dropdown)

On remarque qu'il y a une grosse demande en Mars et en fin d'année à partir de Septembre.
Après on pourrait noter que le nombre de fourniture augmente avec le temps.


In [ ]:
# Agréger les ventes par État et trier décroissant
state_sales = df_profil_year_month.groupby('State')['Sales'].sum().sort_values(ascending=False)

# Bar chart
plt.figure(figsize=(12, 6))
plt.bar(state_sales.index, state_sales.values, color='skyblue')
plt.title('Total Sales by State (Sorted)')
plt.xlabel('State')
plt.ylabel('Total Sales ($)')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 1. Agréger les ventes par État
state_sales = df_profil_year_month.groupby('State')['Sales'].sum().sort_values(ascending=False)

# 2. Fonction d'affichage du bar chart vertical
def plot_top_n_states_vertical(top_n):
    top_states = state_sales.head(top_n)
    
    plt.figure(figsize=(12, 6))
    plt.bar(top_states.index, top_states.values, color='salmon')
    plt.title(f'Top {top_n} States by Total Sales')
    plt.xlabel('State')
    plt.ylabel('Total Sales ($)')
    plt.xticks(rotation=90)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# 3. Slider interactif
slider = widgets.IntSlider(
    value=10,
    min=1,
    max=len(state_sales),
    step=1,
    description='Top N States:',
    continuous_update=False
)

# 4. Lier l’interaction
widgets.interact(plot_top_n_states_vertical, top_n=slider)
